In [2]:
import numpy as np 
import pandas as pd 
import geopandas as gpd
from shapely.wkt import loads
import matplotlib.pyplot as plt
%matplotlib inline

import util.haversine as haversine

In [3]:
### numbers of vehicles in fire frontier if no vehicles move
fire_speed = 1
fire_frontier = pd.read_csv(open('projects/berkeley/demand_inputs/fire_fitted_ellipse.csv'))
fire_frontier['t'] = (fire_frontier['t']-900)/fire_speed ### suppose fire starts at 11.15am
fire_frontier = gpd.GeoDataFrame(fire_frontier, crs={'init':'epsg:4326'}, geometry=fire_frontier['geometry'].map(loads))
print(fire_frontier.loc[fire_frontier['t'].idxmax(), 'geometry'])

def fire_distance(veh_loc, t):
    [veh_o_id, veh_lon, veh_lat] = zip(*veh_loc)
    if t>=np.max(fire_frontier['t']):
        fire_frontier_now = fire_frontier.loc[fire_frontier['t'].idxmax(), 'geometry']
        veh_fire_dist = haversine.point_to_vertex_dist(veh_lon, veh_lat, fire_frontier_now)
    else:
        t_before = np.max(fire_frontier.loc[fire_frontier['t']<=t, 't'])
        t_after = np.min(fire_frontier.loc[fire_frontier['t']>t, 't'])
        fire_frontier_before = fire_frontier.loc[fire_frontier['t']==t_before, 'geometry'].values[0]
        fire_frontier_after = fire_frontier.loc[fire_frontier['t']==t_after, 'geometry'].values[0]
        veh_fire_dist_before = haversine.point_to_vertex_dist(veh_lon, veh_lat, fire_frontier_before)
        veh_fire_dist_after = haversine.point_to_vertex_dist(veh_lon, veh_lat, fire_frontier_after)
        veh_fire_dist = veh_fire_dist_before * (t_after-t)/(t_after-t_before) + veh_fire_dist_after * (t-t_before)/(t_after-t_before)
#     print(np.unique([veh_o_id[i] for i in range(len(veh_fire_dist)) if veh_fire_dist[i]<0]))
    return np.mean(veh_fire_dist), np.sum(veh_fire_dist<0)

nodes = pd.read_csv(open('projects/berkeley/network_inputs/osm_nodes.csv'))
od = pd.read_csv(open('projects/berkeley/demand_inputs/od.csv'))
node_coord_dict = {getattr(row, 'node_osmid'): [getattr(row, 'lon'), getattr(row, 'lat')] for row in nodes.itertuples()}
od_veh_loc = [[getattr(row, 'o_osmid')] + node_coord_dict[getattr(row, 'o_osmid')] for row in od.itertuples()]
fire_frontier_ref = []
for t in range(0, 12000, 10):
    if t%500 == 0: print(t)
    fire_dist_t = fire_distance(od_veh_loc, t)
    fire_frontier_ref.append([t, fire_dist_t[0], fire_dist_t[1]])
fire_frontier_ref_df = pd.DataFrame(fire_frontier_ref, columns=['t', 'avg_fdist', 'neg_fdist'])
print(fire_frontier_ref_df.head())
print(fire_frontier_ref_df.tail())
fire_frontier_ref_df.to_csv('projects/berkeley/demand_inputs/fdist_ref.csv', index=False)

POLYGON ((-122.2407279323787 37.893702330125, -122.2404420871952 37.89908159994273, -122.2416828294798 37.90406173866243, -122.2443159601947 37.90810303091866, -122.2480562216661 37.91076746230701, -122.252498174598 37.91176621781662, -122.2571601859552 37.91099101426586, -122.26153673893 37.90852585664912, -122.2651533467311 37.9046379350266, -122.2676180631413 37.89974864837158, -122.2686639688112 37.89438790469149, -122.2681780238983 37.88913666440806, -122.2662131909996 37.88456396717114, -122.2629825681495 37.88116527340928, -122.2588362214503 37.87930879977119, -122.2542232593414 37.87919565276334, -122.2496432543264 37.88083806572945, -122.2455922386266 37.88405808492575, -122.2425090703845 37.88850684213438, -122.2407279323787 37.893702330125))
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
    t    avg_fdist  neg_fdist
0   0  2569.139320          0
1  10  2555.925180          0
2  20  2542.711041         